<a href="https://colab.research.google.com/github/holezekki99/JODA2022/blob/main/Goalie_salary_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goalie salary predictor

In this workbook we're trying to make a model that could predict what would be the fair salary for NHL goalie based on his performance.

The statistical data for goelies performance is downloaded from Moneypuck.com website. The earnings are webscraped from spotrac.com.

### Aquiring data

We need goalie game statistics first. Separate files for every season. We'll be looking 3 year history:

In [399]:
# Create data-directory
!mkdir ./data

mkdir: cannot create directory ‘./data’: File exists


In [400]:
years = ['2019', '2020', '2021']

In [421]:
# Get the data for seasons starting 2019-2021 and write them to files
import pandas as pd
import requests

for i in years:
  year = str(i)
  url = ('https://moneypuck.com/moneypuck/playerData/seasonSummary/' + year + '/regular/goalies.csv')
  page = requests.get(url)
  r = requests.get(url)
  fname = ('./data/stat' + year + '.csv')
  open(fname, 'wb').write(r.content)


Next step is to webscrape salary information for seasons 2018-2022 and write the result into salaries.csv file.

We use beautifulsoup library for scraping:

In [422]:
from bs4 import BeautifulSoup

# Defining of the dataframe, we collect only name, year and caphit
df = pd.DataFrame(columns=['name', 'season', 'caphit'])

for i in years:
  year = str(i)
  url = ('https://www.spotrac.com/nhl/positional/' + year + '/goaltender/active-cap/')
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  table = soup.find_all('table')[1]

  table = soup.find_all('table')[1] # Table 1 is the list of goalies
  # Collecting Ddata
  for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    if(columns != []):
      # We'll just use the familyname, first name might differ in two data sources
      name = columns[2].text.strip()
      name = name.split()[-1]
      if name == 'Vasilevski':
        name = 'Vasilevskiy'
      year = year
      caphit = columns[3].text.strip()
      df = df.append({'name': name,  'season': year, 'caphit': caphit}, ignore_index=True)


In [423]:
df.to_csv('./data/salaries.csv', index=False)  

### Data cleaning
Next step is to merge the stats-datafiles into a single dataframe, investigate what we have and clean the data if needed:

We start by merging all stat-files into stat_df dataframe:

In [424]:
# Make one dataframe out of multiple statistical files (https://www.geeksforgeeks.org/how-to-merge-multiple-csv-files-into-a-single-pandas-dataframe/)

# importing libraries
import glob
import os
  
# merging the files
stat_files = os.path.join("./data", "stat*.csv")
  
# A list of all joined files is returned
stats_list = glob.glob(stat_files)
  
# Finally, the files are joined
stat_df = pd.concat(map(pd.read_csv, stats_list), ignore_index=True)

# Use only family name. Datasources seem to have differencies in firstnames. (https://pandas.pydata.org/docs/getting_started/intro_tutorials/10_text_data.html)
stat_df['name'] = stat_df['name'].str.split().str.get(1)
stat_df.head()

,playerId,season,name,team,position,situation,games_played,icetime,xGoals,goals,...,highDangerShots,lowDangerxGoals,mediumDangerxGoals,highDangerxGoals,lowDangerGoals,mediumDangerGoals,highDangerGoals,blocked_shot_attempts,penalityMinutes,penalties
0,8475156,2021,Koskinen,EDM,G,other,45,3743.0,8.19,7.0,...,7.0,2.16,3.48,2.56,4.0,3.0,0.0,28.0,0.0,0.0
1,8475156,2021,Koskinen,EDM,G,all,45,157718.0,129.16,136.0,...,105.0,45.27,48.39,35.50,49.0,53.0,34.0,528.0,4.0,2.0
2,8475156,2021,Koskinen,EDM,G,5on5,45,129918.0,93.30,99.0,...,72.0,37.32,35.85,20.13,40.0,43.0,16.0,434.0,2.0,1.0
3,8475156,2021,Koskinen,EDM,G,4on5,45,11732.0,22.73,27.0,...,20.0,5.07,6.57,11.10,5.0,6.0,16.0,64.0,2.0,1.0
4,8475156,2021,Koskinen,EDM,G,5on4,45,12325.0,3.75,2.0,...,5.0,0.61,1.77,1.37,0.0,0.0,2.0,2.0,0.0,0.0


In [425]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   playerId                   1975 non-null   int64  
 1   season                     1975 non-null   int64  
 2   name                       1975 non-null   object 
 3   team                       1975 non-null   object 
 4   position                   1975 non-null   object 
 5   situation                  1975 non-null   object 
 6   games_played               1975 non-null   int64  
 7   icetime                    1975 non-null   float64
 8   xGoals                     1975 non-null   float64
 9   goals                      1975 non-null   float64
 10  unblocked_shot_attempts    1975 non-null   float64
 11  xRebounds                  1975 non-null   float64
 12  rebounds                   1975 non-null   float64
 13  xFreeze                    1975 non-null   float

We have multiple rows for each season per goalie. There is a row for all different game situations (5vs.5, 5vs.4 etc). We're not interested to the difference in different situations, but keep only the "all" rows: 

In [426]:
# remove rows using the drop() function
stat_df.drop(stat_df.index[stat_df['situation'] != 'all'], inplace=True)
stat_df.head()

,playerId,season,name,team,position,situation,games_played,icetime,xGoals,goals,...,highDangerShots,lowDangerxGoals,mediumDangerxGoals,highDangerxGoals,lowDangerGoals,mediumDangerGoals,highDangerGoals,blocked_shot_attempts,penalityMinutes,penalties
1,8475156,2021,Koskinen,EDM,G,all,45,157718.0,129.16,136.0,...,105.0,45.27,48.39,35.50,49.0,53.0,34.0,528.0,4.0,2.0
6,8478499,2021,Hill,SJS,G,all,25,82573.0,56.91,61.0,...,48.0,19.80,20.83,16.29,22.0,21.0,18.0,354.0,0.0,0.0
11,8478009,2021,Sorokin,NYI,G,all,52,184337.0,135.77,123.0,...,122.0,52.35,46.82,36.60,59.0,38.0,26.0,826.0,0.0,0.0
16,8476434,2021,Gibson,ANA,G,all,56,194135.0,157.74,172.0,...,121.0,55.84,60.81,41.07,73.0,74.0,25.0,729.0,12.0,6.0
21,8476341,2021,Forsberg,OTT,G,all,46,154287.0,129.46,121.0,...,117.0,42.33,52.33,34.79,40.0,50.0,31.0,638.0,2.0,1.0


In [427]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 1 to 1971
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   playerId                   395 non-null    int64  
 1   season                     395 non-null    int64  
 2   name                       395 non-null    object 
 3   team                       395 non-null    object 
 4   position                   395 non-null    object 
 5   situation                  395 non-null    object 
 6   games_played               395 non-null    int64  
 7   icetime                    395 non-null    float64
 8   xGoals                     395 non-null    float64
 9   goals                      395 non-null    float64
 10  unblocked_shot_attempts    395 non-null    float64
 11  xRebounds                  395 non-null    float64
 12  rebounds                   395 non-null    float64
 13  xFreeze                    395 non-null    float6

The datatypes in the statististics dataframe are ok. The dataset is good quality, no need for further cleaning.

The next step is to load salary.csv into sal_df:

In [428]:
# Load salaries.csv into a dataframe

sal_df = pd.read_csv('./data/salaries.csv')
sal_df.head()

,name,season,caphit
0,Price,2019,"$10,500,000"
1,Bobrovsky,2019,"$10,000,000"
2,Lundqvist,2019,"$8,500,000"
3,Fleury,2019,"$7,000,000"
4,Rask,2019,"$7,000,000"


In [429]:
sal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    225 non-null    object
 1   season  225 non-null    int64 
 2   caphit  225 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.4+ KB


In [430]:
# Convert year into numeric value:
sal_df['season'] = sal_df['season'].astype(int)

# Convert caphit into numeric value.
sal_df['caphit'] = sal_df['caphit'].replace("[$,]", "", regex=True).astype(int)

In [431]:
sal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    225 non-null    object
 1   season  225 non-null    int64 
 2   caphit  225 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.4+ KB


Now we need to merge the dataframes. Insert into stat_df a new column with salary information ("caphit").

In [415]:
stat_df.shape


(395, 36)

In [416]:
sal_df.shape

(225, 3)

In [432]:
merged_df = pd.merge(stat_df, sal_df, how="left", on=["season", "name"])
merged_df

,playerId,season,name,team,position,situation,games_played,icetime,xGoals,goals,...,lowDangerxGoals,mediumDangerxGoals,highDangerxGoals,lowDangerGoals,mediumDangerGoals,highDangerGoals,blocked_shot_attempts,penalityMinutes,penalties,caphit
0,8475156,2021,Koskinen,EDM,G,all,45,157718.0,129.16,136.0,...,45.27,48.39,35.50,49.0,53.0,34.0,528.0,4.0,2.0,4500000.0
1,8478499,2021,Hill,SJS,G,all,25,82573.0,56.91,61.0,...,19.80,20.83,16.29,22.0,21.0,18.0,354.0,0.0,0.0,NaN
2,8478009,2021,Sorokin,NYI,G,all,52,184337.0,135.77,123.0,...,52.35,46.82,36.60,59.0,38.0,26.0,826.0,0.0,0.0,4000000.0
3,8476434,2021,Gibson,ANA,G,all,56,194135.0,157.74,172.0,...,55.84,60.81,41.07,73.0,74.0,25.0,729.0,12.0,6.0,6400000.0
4,8476341,2021,Forsberg,OTT,G,all,46,154287.0,129.46,121.0,...,42.33,52.33,34.79,40.0,50.0,31.0,638.0,2.0,1.0,900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,8478048,2020,Shesterkin,NYR,G,all,35,113941.0,85.99,83.0,...,28.81,29.87,27.31,30.0,28.0,25.0,443.0,0.0,0.0,925000.0
393,8477992,2020,Johansson,COL,G,all,15,49010.0,30.25,39.0,...,10.65,13.08,6.52,16.0,14.0,9.0,148.0,0.0,0.0,700000.0
394,8481519,2020,Knight,FLA,G,all,4,12414.0,6.90,8.0,...,2.69,1.27,2.94,3.0,1.0,4.0,35.0,0.0,0.0,925000.0
395,8480162,2020,Kivlenieks,CBJ,G,all,2,7401.0,6.00,7.0,...,1.81,1.87,2.31,2.0,2.0,3.0,26.0,0.0,0.0,NaN


Drop goalies that have played less than 25 games in total over 3 seasons:

In [433]:
df = merged_df
df = df[df.groupby('name')['games_played'].transform('sum') > 25]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 0 to 396
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   playerId                   306 non-null    int64  
 1   season                     306 non-null    int64  
 2   name                       306 non-null    object 
 3   team                       306 non-null    object 
 4   position                   306 non-null    object 
 5   situation                  306 non-null    object 
 6   games_played               306 non-null    int64  
 7   icetime                    306 non-null    float64
 8   xGoals                     306 non-null    float64
 9   goals                      306 non-null    float64
 10  unblocked_shot_attempts    306 non-null    float64
 11  xRebounds                  306 non-null    float64
 12  rebounds                   306 non-null    float64
 13  xFreeze                    306 non-null    float64

We see that not all goalies have played in every season.

We can replace goalies NaN caphit values by inserting their average caphit value to those years caphit is missing.

In [436]:
# fill caphit nan values (had not played for that season) with players average caphit (https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group)
df['caphit'] = df['caphit'].fillna(df.groupby('name')['caphit'].transform('mean')) 
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,playerId,season,name,team,position,situation,games_played,icetime,xGoals,goals,...,lowDangerxGoals,mediumDangerxGoals,highDangerxGoals,lowDangerGoals,mediumDangerGoals,highDangerGoals,blocked_shot_attempts,penalityMinutes,penalties,caphit
0,8475156,2021,Koskinen,EDM,G,all,45,157718.0,129.16,136.0,...,45.27,48.39,35.50,49.0,53.0,34.0,528.0,4.0,2.0,4500000.0
1,8478499,2021,Hill,SJS,G,all,25,82573.0,56.91,61.0,...,19.80,20.83,16.29,22.0,21.0,18.0,354.0,0.0,0.0,800000.0
2,8478009,2021,Sorokin,NYI,G,all,52,184337.0,135.77,123.0,...,52.35,46.82,36.60,59.0,38.0,26.0,826.0,0.0,0.0,4000000.0
3,8476434,2021,Gibson,ANA,G,all,56,194135.0,157.74,172.0,...,55.84,60.81,41.07,73.0,74.0,25.0,729.0,12.0,6.0,6400000.0
4,8476341,2021,Forsberg,OTT,G,all,46,154287.0,129.46,121.0,...,42.33,52.33,34.79,40.0,50.0,31.0,638.0,2.0,1.0,900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,8471695,2020,Rask,BOS,G,all,24,83787.0,56.64,53.0,...,17.30,23.59,15.74,19.0,20.0,14.0,257.0,0.0,0.0,7000000.0
392,8478048,2020,Shesterkin,NYR,G,all,35,113941.0,85.99,83.0,...,28.81,29.87,27.31,30.0,28.0,25.0,443.0,0.0,0.0,925000.0
393,8477992,2020,Johansson,COL,G,all,15,49010.0,30.25,39.0,...,10.65,13.08,6.52,16.0,14.0,9.0,148.0,0.0,0.0,700000.0
394,8481519,2020,Knight,FLA,G,all,4,12414.0,6.90,8.0,...,2.69,1.27,2.94,3.0,1.0,4.0,35.0,0.0,0.0,925000.0


Caphit column has 108 NaN values. Let's examine what they are:

In [437]:
nan_values = df[df['caphit'].isna()]
nan_values['name'].unique

<bound method Series.unique of 40      Copley
182    Nilsson
217     Luongo
241       Ward
259    Nilsson
267     Copley
Name: name, dtype: object>

Drop remaining rows with caphit as NaN:

In [438]:
df = df[df['caphit'].notna()]

In [439]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 396
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   playerId                   300 non-null    int64  
 1   season                     300 non-null    int64  
 2   name                       300 non-null    object 
 3   team                       300 non-null    object 
 4   position                   300 non-null    object 
 5   situation                  300 non-null    object 
 6   games_played               300 non-null    int64  
 7   icetime                    300 non-null    float64
 8   xGoals                     300 non-null    float64
 9   goals                      300 non-null    float64
 10  unblocked_shot_attempts    300 non-null    float64
 11  xRebounds                  300 non-null    float64
 12  rebounds                   300 non-null    float64
 13  xFreeze                    300 non-null    float64

Finally the dataframe looks ok.

-----